In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K

beta = 1
n_features = 5
boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences_mix.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0", "M0C", "M1C", "Acceleration"])
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_val = pd.read_csv("Data/Boat_nominal_data/Boat_sequences_mix_val.csv")
boat_val = boat_val.drop(columns=["Unnamed: 0", "M0C", "M1C", "Acceleration"])
scaler = StandardScaler()
val_nom_data = scaler.fit_transform(boat_val)

# boat_anom_csv = pd.read_csv("Data/Boat_anomalous_big.csv")
# boat_anom_csv= boat_anom_csv.drop(columns=["Unnamed: 0"])    
# scaler = StandardScaler()
# anomalous_data = scaler.fit_transform(boat_anom_csv)
# print(anomalous_data.shape)
# 
# boat_anom_val_csv = pd.read_csv("Data/Boat_anomalous_validation.csv")
# boat_anom_val_csv= boat_anom_val_csv.drop(columns=["Unnamed: 0"])    
# scaler = StandardScaler()
# anomalous_val_data = scaler.fit_transform(boat_anom_val_csv)
# print(anomalous_val_data.shape)

def prepare_sequences(data, batch_size):
    samples = []
    for i in range(0,data.shape[0]- batch_size, batch_size):
        sample = data[i:i+batch_size]	
        samples.append(sample)
    sequences = np.array(samples)
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))
    return trainX


def prepare_data():    
    trainX_nominal = prepare_sequences(normal_data,1024) 
    print(trainX_nominal.shape)
    
    valX_nominal = prepare_sequences(val_nom_data,1024)
    print(valX_nominal.shape)

    return trainX_nominal, valX_nominal

trainX_nominal, valX_nominal = prepare_data()


(103424, 5)
(100, 1024, 5)
(10, 1024, 5)


In [21]:
from keras.layers import MaxPooling1D

input_shape = (1024, n_features)
kernel_size = 3
filters = 64
latent_dim = 10
epochs = 30
use_mse = True
load_weights = False


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


inputs = Input(shape=input_shape, name='encoder_input')
x = inputs
for i in range(4):
    x = Conv1D(filters=filters,
               kernel_size=7,
               padding='same')(x)
    x = MaxPooling1D(2)(x)
    filters = int(filters / 2)


shape = K.int_shape(x)

x = Flatten()(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2], name='Dense_after_sampling')(latent_inputs)
x = Reshape((shape[1], shape[2]))(x)
filters = filters * 2

for i in range(4):
    x = Conv1D(filters=filters,kernel_size=7, padding='same')(x)
    x = UpSampling1D(size=2)(x)
    filters = filters * 2
    
    
outputs = Conv1D(filters=n_features, kernel_size=7, padding='same')(x)


decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

from keras.losses import mse

reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
loss = reconstruction_loss + beta*kl_loss
vae.add_loss(loss)

vae.compile(optimizer='rmsprop', metrics= ['accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 1024, 5)      0                                            
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 1024, 64)     2304        encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling1d_17 (MaxPooling1D) (None, 512, 64)      0           conv1d_37[0][0]                  
__________________________________________________________________________________________________
conv1d_38 (Conv1D)              (None, 512, 32)      14368       max_pooling1d_17[0][0]           
__________________________________________________________________________________________________
max_poolin

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 10)                0         
_________________________________________________________________
Dense_after_sampling (Dense) (None, 512)               5632      
_________________________________________________________________
reshape_5 (Reshape)          (None, 64, 8)             0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 64, 8)             456       
_________________________________________________________________
up_sampling1d_17 (UpSampling (None, 128, 8)            0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 128, 16)           912       
_________________________________________________________________
up_sampling1d_18 (UpSampling (None, 256, 16)           0         
__________

In [27]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="Models/Weights/weights.hdf5", verbose=1,
                               save_best_only=True)

vae.fit(x=trainX_nominal, epochs=100, 
        batch_size=1024,
        validation_data=(valX_nominal,None),
        callbacks=[checkpointer])
vae.load_weights('Models/Weights/weights.hdf5')

# vae.save("Models/Conv1d_VAE_comp.h5")

Train on 300 samples, validate on 10 samples
Epoch 1/100


300/300 [==============================] - 3s 9ms/step - loss: 0.1552 - val_loss: 0.1527



Epoch 00001: val_loss improved from inf to 0.15266, saving model to Models/Weights/weights.hdf5
Epoch 2/100


300/300 [==============================] - 3s 9ms/step - loss: 0.1562 - val_loss: 0.1596



Epoch 00002: val_loss did not improve from 0.15266
Epoch 3/100


...


Epoch 00098: val_loss did not improve from 0.12243
Epoch 99/100


300/300 [==============================] - 3s 9ms/step - loss: 0.1260 - val_loss: 0.1248



Epoch 00099: val_loss did not improve from 0.12243
Epoch 100/100


300/300 [==============================] - 3s 8ms/step - loss: 0.1241 - val_loss: 0.1262



Epoch 00100: val_loss did not improve from 0.12243


In [119]:
import sys

orig_stdout = sys.stdout
f = open('Logs/out.txt', 'w')
sys.stdout = f
print(encoder.summary())
print(decoder.summary())
sys.stdout = orig_stdout
f.close()


In [28]:



nom_autoenc = vae.predict(trainX_nominal)

autoenc_df = pd.DataFrame(nom_autoenc[0], columns=boat_csv.columns)

plt.plot(boat_csv['Lon'][:1024], boat_csv['Lat'][:1024])
plt.show()
plt.plot(autoenc_df['Lon'], autoenc_df['Lat'])
plt.show()





In [30]:
nom_enc = encoder.predict(trainX_nominal)
print(len(nom_enc), nom_enc[0].shape)

titles = ["Mean","Sigmas","Sampled"]

for i, elem in enumerate(nom_enc):
    plt.plot(elem)
    plt.title(titles[i])
    plt.show()








3 (300, 10)
